# Usage

## Command line utility

TODO

```bash
$ r5py travel-time-matrix somewhere.osm.pbf somehwere_gtfs.zip --origins grid-centroids.gpkg ...
```

## Python

The entire functionality is exposed via r5py’s [Python API](reference.html).

In [ ]:
# this cell is hidden from output
# it’s used to 
# - import credentials
# - hide the warnings that are displayed because 
#   of a current mismatch in geos versions between 
#   shapely and pygeos
# - set sys.path to point to the local streetviewdownloader repo
# - make ipython show all outputs of each cell
# - provide a helper function to unobstrusively plot geometries


### Load transport network

Virtually all operations of r5py require a transport network. As input files to construct a transport network topology, r5py needs the following data sets:

- The street network in the form of an OpenStreetMap extract covering the study area, prepared according to the instructions at https://docs.conveyal.com/prepare-inputs#preparing-the-osm-data, and possibly annotated with LADOT tags (see https://github.com/RSGInc/ladot_analysis_dataprep).
- Public transport schedules in GTFS format (optional, r5py can combine multiple GTFS files)
- TODO: elevation model

In this example, we use data from Helsinki metropolitan area. TODO: reference data sources (OSM, HSL)

To import the street and public transport networks, instantiate an `r5py.TransportNetwork` with the file paths to the OSM extract and the GTFS files:

In [ ]:
from r5py import TransportNetwork

transport_network = TransportNetwork(
    "data/paakaupunkiseutu.pbf",
    [
        "data/GTFS.zip"
    ]
)

## Compute a travel time matrix

A travel time matrix is a data set detailing the travel costs (e.g., time) from each point to each point in a set of origins and destinations in a study area. 

For the sake of this exercise, we will create a regular grid of points covering parts of Helsinki.

In [ ]:
import itertools
import geopandas
import numpy

north_east = (24.97, 60.22)
south_west = (24.90, 60.16)

coordinates =  itertools.product(
    numpy.arange(south_west[0], north_east[0], 0.01),
    numpy.arange(south_west[1], north_east[1], 0.01)
)

x, y = zip(*coordinates)

grid_points = geopandas.GeoDataFrame({
    "id": range(len(list(x))),
    "geometry": geopandas.GeoSeries.from_xy(
     x,y
    )
})
grid_points.plot()
grid_points.head()

Now, to compute a travel time matrix between all `grid_points`, we first instantiate an `r5py.TravelTimeMatrix` with the transport network we created earlier, and a list of origins (since we don’t specify a separate set of destinations, r5py will use the same points as destinations and produce a full set of origins and destinations). The constructor also accepts all parameters of [RegionalTask](reference.html#r5py.RegionalTask), such as transport modes, or walking speed. 

Calling `compute_travel_times()` on the instance will return a `pandas.DataFrame` with travel times between all points.

In [ ]:
import datetime

from r5py import TravelTimeMatrix, TransitMode, LegMode


travel_time_matrix = TravelTimeMatrix(
    transport_network,
    origins=grid_points,
    departure=datetime.datetime(2022,2,22,8,30),
    transport_modes=[TransitMode.TRANSIT, LegMode.WALK]
)
travel_times = travel_time_matrix.compute_travel_times()
travel_times

In [ ]:
from streetviewdownloader import StreetViewDownloader

downloader = StreetViewDownloader(API_KEY, URL_SIGNING_KEY)
downloader.download(kirkkonummi, output_directory)

This downloads the metadata to `metadata.gpkg`, and all images into the output directory. Note that, by default, data is appended to an existing metadata dataset, and images existing in the directory are not downloaded again. If you want the metadata only, call `download()` with `metadata_only=True`.

Depending on the location and size of the extent, this operation can take a while. In the background, Streetviewdownloader carries out multiple steps:

1. Decide which OSM extract from either geofabrik.de or bbbike.org fits best for the extent (the smallest extract in which the extent is fully contained)
2. Download this OSM extract, and extract and clip the street network to the extent
3. Generate a point data set on this street network with at most 25 meter distance between points
4. Query the StreetView Static API for panoramas in the immediate vicinity of each point (and remove duplicates afterwards)
5. Download six images each for every panorama.

The results of the first two steps are cached for up to seven days, as they are the most expensive ones in respect to download time and processing time. They should complete fast in subsequent runs.